# Face Mask Detection using a Convolutional Neural Network (CNN)

This notebook walks through the process of building a CNN model to classify whether a person in an image is wearing a face mask.

### Step 1: Setup and Dataset Download

In [ ]:
# kaggle installation
!pip install kaggle

In [ ]:
# Setting up the Kaggle API credentials.
# This is just boilerplate to get our kaggle.json file in the right place.
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
# download the dataset from Kaggle.
!kaggle datasets download -d omkargurav/face-mask-dataset

In [ ]:

from zipfile import ZipFile
dataset = '/content/face-mask-dataset.zip'

with ZipFile(dataset,'r') as zip:
  zip.extractall()
  print('Dataset extraction successful!')

### Step 2: Import Libraries and Explore Data

In [ ]:
# Importing all the good stuff we need for this project.
# Standard libraries for data handling, plotting, and of course, deep learning.
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import cv2
from PIL import Image
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models

In [ ]:
# Getting a list of all the image filenames from both directories.
with_mask_files = os.listdir('/content/data/with_mask')
without_mask_files = os.listdir('/content/data/without_mask')

In [ ]:
# Just a quick check to see how many images we have for each class.
print("Number of images with mask: ", len(with_mask_files))
print("Number of images without mask: ", len(without_mask_files))

### Step 3: Data Pre-processing

In [ ]:
# Creating numerical labels for our two classes.
# This is how the model will understand the difference.
# With Mask --> 1
# Without Mask --> 0
with_mask_labels = [1] * len(with_mask_files)
without_mask_labels = [0] * len(without_mask_files)

# Combining the labels into a single list.
labels = with_mask_labels + without_mask_labels

In [ ]:

# Looping through each image, resizing it to 128x128, and converting it into a NumPy array.
with_mask_path = '/content/data/with_mask/'
without_mask_path = '/content/data/without_mask/'
data = []

# Processing 'with_mask' images
for eachImg in with_mask_files:
    image = Image.open(with_mask_path + eachImg)
    image = image.resize((128, 128))
    image = image.convert('RGB') 
    image = np.array(image)
    data.append(image)

# Processing 'without_mask' images
for eachImg in without_mask_files:
    image = Image.open(without_mask_path + eachImg)
    image = image.resize((128, 128))
    image = image.convert('RGB')
    image = np.array(image)
    data.append(image)

In [ ]:
# Converting our lists of data and labels into NumPy arrays.
# This is the format TensorFlow/Keras expects.
X = np.array(data)
Y = np.array(labels)

In [ ]:
# Splitting the data into training and testing sets.
# 80% for training, 20% for testing.
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=2)

In [ ]:
# Normalizing the pixel values.
# Scaling image data from [0, 255] to [0, 1] helps the model train faster and more effectively.
x_train = x_train / 255.0
x_test = x_test / 255.0

### Step 4: Build and Train the CNN Model

In [ ]:
# Building the CNN model.
numClasses = 2

model = keras.Sequential()

# First convolutional block: 32 filters, 3x3 kernel, followed by a max pooling layer.

model.add(layers.Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(128, 128, 3)))
model.add(layers.MaxPool2D(pool_size=(2, 2)))

# Second convolutional block: 64 filters. 
model.add(layers.Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(layers.MaxPool2D(pool_size=(2, 2)))

# Flatten the output from the convolutional layers to feed it into the dense layers.
model.add(layers.Flatten())

# Adding some dense layers for classification.
# Dropout helps prevent overfitting by randomly dropping neurons during training.
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dropout(0.5))

model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dropout(0.5))

# The final output layer. Sigmoid is used for binary classification.
model.add(layers.Dense(numClasses, activation='sigmoid'))

In [ ]:
# Compiling the model.

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
# Training the model for 5 epochs.
history = model.fit(x_train, y_train, epochs=5)

### Step 5: Evaluate the Model

In [ ]:

loss, accuracy = model.evaluate(x_test, y_test)
print(f"Test accuracy: {accuracy*100:.2f}% | Test loss: {loss:.4f}")

### Step 6: Test with a Custom Image

In [ ]:
# a function to predict on any custom image.

class_names = ['Without Mask', 'With Mask'] # Correcting order to match labels (0 and 1)

def predict_custom_image(model, img_path):
  # 1. Load and resize the image
  img = Image.open(img_path)
  img = img.resize((128, 128))

  # 2. Convert to NumPy array and normalize
  img_array = np.array(img)
  img_array = img_array / 255.0

  # 3. Add a batch dimension, because the model expects it
  img_batch = np.expand_dims(img_array, axis=0)

  # 4. Make a prediction
  prediction = model.predict(img_batch)
  pred_class_index = np.argmax(prediction[0])
  confidence = np.max(prediction[0]) * 100
  predicted_class_name = class_names[pred_class_index]

  # 5. Display the result
  plt.figure(figsize=(6, 6))
  plt.imshow(img)
  plt.title(f"Predicted: {predicted_class_name}\nConfidence: {confidence:.2f}%")
  plt.axis('off')
  plt.show()

  print(f"The model predicted this image is: {predicted_class_name}")

In [ ]:

imgPath = '/content/custom_datasets/img6.jpg'
predict_custom_image(model, imgPath)